In [130]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [131]:
df_t = pd.read_csv('../data/raw/training_Traffic.csv')
df_s = pd.read_csv('../data/raw/training_Sales.csv')

In [132]:
# increasing max length for all columns and rows
pd.set_option('display.max_colwidth', -1)

pd.set_option('display.max_info_columns', 500)
pd.set_option('display.max_rows', 500)

In [133]:
df_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58437 entries, 0 to 58436
Data columns (total 2 columns):
Date     58437 non-null object
Value    58437 non-null int64
dtypes: int64(1), object(1)
memory usage: 913.2+ KB


In [134]:
df_s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67058 entries, 0 to 67057
Data columns (total 2 columns):
Date     67058 non-null object
Value    67058 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


In [135]:
# convert date into datetime columns
df_t['Date'] = pd.to_datetime(df_t['Date'])
df_s['Date'] = pd.to_datetime(df_s['Date'])

In [136]:
# setting date as indexes
df_t.set_index('Date', inplace=True)
df_s.set_index('Date', inplace=True)

In [137]:
df_t.head(5)

,Value
Date,
2015-01-01 10:00:00,0
2015-01-01 10:15:00,1
2015-01-01 10:30:00,4
2015-01-01 10:45:00,3
2015-01-01 11:00:00,1


In [138]:
df_s.head(5)

,Value
Date,
2013-11-07 13:00:00,830
2013-11-07 13:15:00,885
2013-11-07 13:30:00,938
2013-11-07 13:45:00,804
2013-11-07 14:00:00,793


There are no missing values in the current datasets for the dates provided

In [139]:
# checking for missing values in 
df_t.isna().sum()

Value    0
dtype: int64

In [140]:
df_s.isna().sum()

Value    0
dtype: int64

The datasets provided are too granular. Since we are interested only in predicting traffic and sales by the hour, it is possible to do some downsampling on the dates

In [141]:
# downsampling by the hour for all traffic datapoints
temp_hourly_traffic = df_t.resample('H')
df_t_h = temp_hourly_traffic.sum()

In [142]:
# downsampling by the hour for all sales datapoints
temp_hourly_sales = df_s.resample('H')
df_s_h = temp_hourly_sales.sum()

In [143]:
df_t_h.head()

,Value
Date,
2015-01-01 10:00:00,8
2015-01-01 11:00:00,14
2015-01-01 12:00:00,28
2015-01-01 13:00:00,33
2015-01-01 14:00:00,47


In [144]:
df_s_h.head(5)

,Value
Date,
2013-11-07 13:00:00,3457
2013-11-07 14:00:00,2250
2013-11-07 15:00:00,0
2013-11-07 16:00:00,729
2013-11-07 17:00:00,0


Double checking that there are no missing hours/dates

In [145]:
pd.period_range(start = df_t_h.index.min(), end = df_t_h.index.max(), freq = 'H')

PeriodIndex(['2015-01-01 10:00', '2015-01-01 11:00', '2015-01-01 12:00',
             '2015-01-01 13:00', '2015-01-01 14:00', '2015-01-01 15:00',
             '2015-01-01 16:00', '2015-01-01 17:00', '2015-01-01 18:00',
             '2015-01-01 19:00',
             ...
             '2018-05-06 12:00', '2018-05-06 13:00', '2018-05-06 14:00',
             '2018-05-06 15:00', '2018-05-06 16:00', '2018-05-06 17:00',
             '2018-05-06 18:00', '2018-05-06 19:00', '2018-05-06 20:00',
             '2018-05-06 21:00'],
            dtype='period[H]', length=29316, freq='H')

In [146]:
pd.period_range(start = '2015-01-01 10:00', end = '2018-05-06 21:00', freq='H')

PeriodIndex(['2015-01-01 10:00', '2015-01-01 11:00', '2015-01-01 12:00',
             '2015-01-01 13:00', '2015-01-01 14:00', '2015-01-01 15:00',
             '2015-01-01 16:00', '2015-01-01 17:00', '2015-01-01 18:00',
             '2015-01-01 19:00',
             ...
             '2018-05-06 12:00', '2018-05-06 13:00', '2018-05-06 14:00',
             '2018-05-06 15:00', '2018-05-06 16:00', '2018-05-06 17:00',
             '2018-05-06 18:00', '2018-05-06 19:00', '2018-05-06 20:00',
             '2018-05-06 21:00'],
            dtype='period[H]', length=29316, freq='H')

There are no missing hours for our traffic hourly dataframe

In [147]:
pd.period_range(start = df_s_h.index.min(), end = df_s_h.index.max(), freq = 'H')

PeriodIndex(['2013-11-07 13:00', '2013-11-07 14:00', '2013-11-07 15:00',
             '2013-11-07 16:00', '2013-11-07 17:00', '2013-11-07 18:00',
             '2013-11-07 19:00', '2013-11-07 20:00', '2013-11-07 21:00',
             '2013-11-07 22:00',
             ...
             '2018-05-06 10:00', '2018-05-06 11:00', '2018-05-06 12:00',
             '2018-05-06 13:00', '2018-05-06 14:00', '2018-05-06 15:00',
             '2018-05-06 16:00', '2018-05-06 17:00', '2018-05-06 18:00',
             '2018-05-06 19:00'],
            dtype='period[H]', length=39391, freq='H')

In [148]:
pd.period_range(start = '2013-11-07 13:00', end = '2018-05-06 19:00', freq='H')

PeriodIndex(['2013-11-07 13:00', '2013-11-07 14:00', '2013-11-07 15:00',
             '2013-11-07 16:00', '2013-11-07 17:00', '2013-11-07 18:00',
             '2013-11-07 19:00', '2013-11-07 20:00', '2013-11-07 21:00',
             '2013-11-07 22:00',
             ...
             '2018-05-06 10:00', '2018-05-06 11:00', '2018-05-06 12:00',
             '2018-05-06 13:00', '2018-05-06 14:00', '2018-05-06 15:00',
             '2018-05-06 16:00', '2018-05-06 17:00', '2018-05-06 18:00',
             '2018-05-06 19:00'],
            dtype='period[H]', length=39391, freq='H')

There are no missing hours for our sales hourly dataframe

# EDA Visualisation

In [155]:
# building more detailed df
df_t_h['time'] = df_t_h.index.hour
df_t_h['mdate'] = df_t_h.index.day
df_t_h['day'] = df_t_h.index.dayofweek
df_t_h['week'] = df_t_h.index.weekofyear
df_t_h['month'] = df_t_h.index.month
df_t_h['year'] = df_t_h.index.year

In [156]:
df_t_h

,Value,time,mdate,day,week,month,year
Date,,,,,,,
2015-01-01 10:00:00,8,10,1,3,1,1,2015
2015-01-01 11:00:00,14,11,1,3,1,1,2015
2015-01-01 12:00:00,28,12,1,3,1,1,2015
2015-01-01 13:00:00,33,13,1,3,1,1,2015
2015-01-01 14:00:00,47,14,1,3,1,1,2015
...,...,...,...,...,...,...,...
2018-05-06 17:00:00,27,17,6,6,18,5,2018
2018-05-06 18:00:00,14,18,6,6,18,5,2018
2018-05-06 19:00:00,4,19,6,6,18,5,2018


In [157]:
df_s_h['time'] = df_s_h.index.hour
df_s_h['mdate'] = df_s_h.index.day
df_s_h['day'] = df_s_h.index.dayofweek
df_s_h['week'] = df_s_h.index.weekofyear
df_s_h['month'] = df_s_h.index.month
df_s_h['year'] = df_s_h.index.year

In [158]:
df_s_h

,Value,time,mdate,day,week,month,year
Date,,,,,,,
2013-11-07 13:00:00,3457,13,7,3,45,11,2013
2013-11-07 14:00:00,2250,14,7,3,45,11,2013
2013-11-07 15:00:00,0,15,7,3,45,11,2013
2013-11-07 16:00:00,729,16,7,3,45,11,2013
2013-11-07 17:00:00,0,17,7,3,45,11,2013
...,...,...,...,...,...,...,...
2018-05-06 15:00:00,3645,15,6,6,18,5,2018
2018-05-06 16:00:00,3372,16,6,6,18,5,2018
2018-05-06 17:00:00,3077,17,6,6,18,5,2018


In [168]:
# Prepare data for variation by hour of the day (total)
pivothour_t = pd.pivot_table(df_t_h, values='Value', columns='day',
                         aggfunc=np.nanmean)
dfdays_t = pd.DataFrame(pivothour_t, index=range(0,24), columns=range(0,7))

In [169]:
pivothour_t

day,0,1,2,3,4,5,6
Value,6.664272,5.933908,6.08501,6.083055,9.530238,14.049524,10.338971
